In [1]:
from cartopy.io.shapereader import Reader
from bokeh.models import ColumnDataSource, HoverTool, LogColorMapper, WidgetBox, ColorMapper
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, Figure
from bokeh.layouts import column
from bokeh.palettes import Viridis256 as palette
#from bokeh.palettes import Magma256 as palette
from bokeh.models.widgets import Panel, Tabs, Slider, RadioButtonGroup
import os
import pandas as pd
import pickle
import numpy as np
from ipywidgets import interact, SelectionSlider, Select, ToggleButtons

output_notebook()

Loading BokehJS ...

In [2]:
#read the pickled data
pickle_in = open(os.path.join(os.getcwd(), 'data', 'processed', 'all_data.pickle'), 'rb')
data = pickle.load(pickle_in)

#keep a copy of the original data just in case
data_orig = data.copy()

income_scorer = np.array([5, 12.5, 17.5, 22.5, 27.5, 32.5, 37.5, 42.5, 47.5,
                          55, 67.5, 87.5, 112.5, 137.5, 175, 300])
rent_scorer = np.array([50, 125, 175, 225, 275, 325, 375, 425, 475, 525, 575,
                        625, 675, 725, 775, 850, 950, 1125, 1375, 1750, 2500])
value_scorer = np.array([5, 12.5, 17.5, 22.5, 27.5, 32.5, 27.5, 45, 55, 65, 75, 
                         85, 95, 112.5, 137.5, 162.5, 187.5, 225, 275, 350, 450, 
                         625, 875, 1500])
education_scorer = np.arange(1, 8)

for year in data.keys():
    for key in data[year].keys():
        tmp = data[year][key]
        #bust out the total population and total households from the race and families df
        if key == 'race':
            population = tmp['total_population'].to_frame()
        #convert columns to percentages and drop the total
        tmp = tmp[tmp.columns[1:]].divide(tmp[tmp.columns[0]], axis=0).fillna(0)
        #calculate the index (single number to represent bins) 
        if len(tmp.columns) > 1:
            if key == 'income':
                tmp['income_index'] = tmp.dot(income_scorer)
            elif key == 'rent':
                tmp['rent_index'] = tmp.dot(rent_scorer)
            elif key == 'value':
                tmp['value_index'] = tmp.dot(value_scorer)
            elif key == 'education':
                tmp['education_index'] = tmp.dot(education_scorer)
        #put the new percentage dataframe back in
        data[year][key] = tmp
    #add the busted out totals to the data dict
    data[year]['population'] = population

In [3]:
#read the shapefile
tracts = Reader('census_tract_shapefile/austin_metro_tracts.shp')
#extract x's and y's
xs, ys, names, tract = [], [], [], []
for r, g in zip(tracts.records(), tracts.geometries()):
    #skip the airport tract - all zeros and nans
    if r.attributes['GEOID'] not in ['48453980000', '48453001606', '48453002319']:
        xs.append(list(g[0].exterior.coords.xy[0]))
        ys.append(list(g[0].exterior.coords.xy[1]))
        names.append(int(r.attributes['GEOID']))
        tract.append(float(r.attributes['GEOID'][-6:]) / 100)

In [4]:
color_mapper = LogColorMapper(palette=palette)

TOOLS = "pan,wheel_zoom,reset,hover,save"

In [5]:
def make_plot(year):
    popl = data[year]['population'].loc[names]['total_population'].tolist()
    income = data[year]['income'].loc[names]['income_index'].tolist()
    education = data[year]['education'].loc[names]['education_index'].tolist()
    race = data[year]['race'].loc[names]['white_alone'].tolist()
    rent = data[year]['rent'].loc[names]['rent_index'].tolist()
    unemployment = data[year]['unemployment'].loc[names]['unemployed'].tolist()
    value = data[year]['value'].loc[names]['value_index'].tolist()
    years = [year] * len(names)
    
    source = ColumnDataSource(data=dict(
        x=xs,
        y=ys,
        name=names,
        tract=tract,
        years=years,
        popl=popl,
        income=income,
        education=education,
        race=race,
        rent=rent,
        unemployment=unemployment,
        value=value,
    ))
    
    p_pop = figure(title='Total Population - {}'.format(year), tools=TOOLS, x_axis_location=None,
                   y_axis_location=None)
    p_pop.patches('x', 'y', source=source, fill_color={'field': 'popl', 'transform': color_mapper},
                  fill_alpha=0.7, line_color='black', line_width=0.5)
    hover_pop = p_pop.select_one(HoverTool)
    hover_pop.point_policy = 'follow_mouse'
    hover_pop.tooltips = [
        ('Year', '@years'),
        ('GeoID', '@name'),
        ('Census Tract', '@tract'),
        ('Lon, Lat', '$x, $y'),
        ('Population', '@popl')
    ]
    tab_pop = Panel(child=p_pop, title='Population')
    
    p_race = figure(title='Percentage White - {}'.format(year), tools=TOOLS, x_axis_location=None,
                    y_axis_location=None)
    p_race.patches('x', 'y', source=source, fill_color={'field': 'race', 'transform': color_mapper},
                   fill_alpha=0.7, line_color='black', line_width=0.5)
    hover_race = p_race.select_one(HoverTool)
    hover_race.point_policy = 'follow_mouse'
    hover_race.tooltips = [
        ('Year', '@years'),
        ('GeoID', '@name'),
        ('Census Tract', '@tract'),
        ('Lon, Lat', '$x, $y'),
        ('Percentage White', '@race')
    ]
    tab_race = Panel(child=p_race, title='Race')
    
    p_income = figure(title='Income - {}'.format(year), tools=TOOLS, x_axis_location=None,
                      y_axis_location=None)
    p_income.patches('x', 'y', source=source, fill_color={'field': 'income', 'transform': color_mapper},
                     fill_alpha=0.7, line_color='black', line_width=0.5)
    hover_income = p_income.select_one(HoverTool)
    hover_income.point_policy = 'follow_mouse'
    hover_income.tooltips = [
        ('Year', '@years'),
        ('GeoID', '@name'),
        ('Census Tract', '@tract'),
        ('Lon, Lat', '$x, $y'),
        ('Income Index', '@income')
    ]
    tab_income = Panel(child=p_income, title='Income')
    
    p_education = figure(title='Education Attained - {}'.format(year), tools=TOOLS, x_axis_location=None,
                         y_axis_location=None)
    p_education.patches('x', 'y', source=source, fill_color={'field': 'education', 'transform': color_mapper},
                        fill_alpha=0.7, line_color='black', line_width=0.5)
    hover_education = p_education.select_one(HoverTool)
    hover_education.point_policy = 'follow_mouse'
    hover_education.tooltips = [
        ('Year', '@years'),
        ('GeoID', '@name'),
        ('Census Tract', '@tract'),
        ('Lon, Lat', '$x, $y'),
        ('Education Index', '@education')
    ]
    tab_education = Panel(child=p_education, title='Education')
    
    p_unemployment = figure(title='Percentage Unemployed - {}'.format(year), tools=TOOLS, x_axis_location=None,
                            y_axis_location=None)
    p_unemployment.patches('x', 'y', source=source, fill_color={'field': 'unemployment', 'transform': color_mapper},
                           fill_alpha=0.7, line_color='black', line_width=0.5)
    hover_unemployment = p_unemployment.select_one(HoverTool)
    hover_unemployment.point_policy = 'follow_mouse'
    hover_unemployment.tooltips = [
        ('Year', '@years'),
        ('GeoID', '@name'),
        ('Census Tract', '@tract'),
        ('Lon, Lat', '$x, $y'),
        ('Percentage Unemployed', '@unemployment')
    ]
    tab_unemployment = Panel(child=p_unemployment, title='Unemployment')
    
    p_rent = figure(title='Rent - {}'.format(year), tools=TOOLS, x_axis_location=None,
                    y_axis_location=None)
    p_rent.patches('x', 'y', source=source, fill_color={'field': 'rent', 'transform': color_mapper},
                  fill_alpha=0.7, line_color='black', line_width=0.5)
    hover_rent = p_rent.select_one(HoverTool)
    hover_rent.point_policy = 'follow_mouse'
    hover_rent.tooltips = [
        ('Year', '@years'),
        ('GeoID', '@name'),
        ('Census Tract', '@tract'),
        ('Lon, Lat', '$x, $y'),
        ('Rent Index', '@rent')
    ]
    tab_rent = Panel(child=p_rent, title='Rent')
    
    p_value = figure(title='Home Value - {}'.format(year), tools=TOOLS, x_axis_location=None,
                     y_axis_location=None)
    p_value.patches('x', 'y', source=source, fill_color={'field': 'value', 'transform': color_mapper},
                    fill_alpha=0.7, line_color='black', line_width=0.5)
    hover_value = p_value.select_one(HoverTool)
    hover_value.point_policy = 'follow_mouse'
    hover_value.tooltips = [
        ('Year', '@years'),
        ('GeoID', '@name'),
        ('Census Tract', '@tract'),
        ('Lon, Lat', '$x, $y'),
        ('Home Value Index', '@value')
    ]
    tab_value = Panel(child=p_value, title='Home Value')
    
    tabs = Tabs(tabs=[tab_pop, tab_race, tab_income, tab_education,
                      tab_unemployment, tab_rent, tab_value])
    
    show(tabs)

In [6]:
toggle = ToggleButtons(
    options=['2000', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
    value='2000',
    description='Year:'
)

interact(make_plot, year=toggle);

interactive(children=(ToggleButtons(description='Year:', options=('2000', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016'), value='2000'), Output()), _dom_classes=('widget-interact',))

In [7]:
data

{'2000': {'education':                    <9      9-12  high_school  some_college  associate  \
  geoid                                                                   
  48021950100  0.088794  0.113127     0.352615      0.202348   0.062327   
  48021950200  0.185519  0.171038     0.316393      0.148361   0.033333   
  48021950300  0.081940  0.103954     0.320220      0.272727   0.054219   
  48021950400  0.073967  0.084711     0.255372      0.254545   0.041736   
  48021950600  0.096296  0.136232     0.341063      0.241868   0.039291   
  48021950700  0.114307  0.152166     0.323262      0.189661   0.057153   
  48055960200  0.119908  0.229304     0.401774      0.165900   0.025953   
  48055960300  0.089792  0.088522     0.310462      0.232952   0.038119   
  48055960400  0.185103  0.170384     0.362177      0.176182   0.016503   
  48055960500  0.119316  0.167226     0.328349      0.203845   0.036009   
  48055960600  0.051958  0.142319     0.357681      0.228163   0.055723   
  48